# Projet GPT - Train

 This is a smal project where i tried to create my own simple llm 

In [10]:
import torch, torch.nn as nn, torch.optim as optim
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import random, math

from datasets import load_dataset

## Dataset

In [ ]:
wikitext2 = load_dataset(
    "Salesforce/wikitext",    
    name="wikitext-2-raw-v1",   
    split="train"                
)

  = valkyria chronicles iii = 
   senjō no valkyria 3 : unrecorded chronicles ( japanese : 戦場のヴァルキュリ
Train corpus size: 2051910 tokens
Corpus size: 10929708 characters


In [ ]:
corpus = " ".join(rec["text"] for rec in wikitext2).lower()